In [1]:
import keras
keras.__version__


'2.4.3'

#### learning word embeddings with `EMBEDDING` layer

In [2]:
from keras.layers import Embedding

# The Embedding layer takes at least two arguments:
# the number of possible tokens, here 1000 (1 + maximum word index),
# and the dimensionality of the embeddings, here 64.
embedding_layer = Embedding(1000, 64)

# good explanation: 
# https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work

In [7]:
from keras.datasets import imdb
from keras import preprocessing

# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words 
# (among top max_features most common words)
maxlen = 20

# Load the data as lists of integers.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# This turns our lists of integers
# into a 2D integer tensor of shape `(samples, maxlen)`
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [10]:
x_train

array([[  65,   16,   38, ...,   19,  178,   32],
       [  23,    4, 1690, ...,   16,  145,   95],
       [1352,   13,  191, ...,    7,  129,  113],
       ...,
       [  11, 1818, 7561, ...,    4, 3586,    2],
       [  92,  401,  728, ...,   12,    9,   23],
       [ 764,   40,    4, ...,  204,  131,    9]], dtype=int32)

In [12]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
model.add(Embedding(10000, 8, input_length=maxlen))
# After the Embedding layer, 
# our activations have shape `(samples, maxlen, 8)`.


# 8 = output_dimension of the new vector for each word for which embedding layer will be trained
# max_length = max size of each input sentence

# so once the embeddinglayer is trained it creates an output of output * input. Then after fitting the model with this 
# embedding layer, the final shape of the weight matrix will be 
# => number of unique words in the vocab * size of word vector

# https://medium.com/analytics-vidhya/understanding-embedding-layer-in-keras-bbe3ff1327ce

In [13]:
# We flatten the 3D tensor of embeddings 
# into a 2D tensor of shape `(samples, maxlen * 8)`
model.add(Flatten())

# We add the classifier on top
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 1s 810us/step - loss: 0.6704 - acc: 0.6208 - val_loss: 0.6219 - val_acc: 0.6998
Epoch 2/10
625/625 [==============================] - 0s 653us/step - loss: 0.5451 - acc: 0.7501 - val_loss: 0.5266 - val_acc: 0.7304
Epoch 3/10
625/625 [==============================] - 0s 637us/step - loss: 0.4622 - acc: 0.7875 - val_loss: 0.4

In [20]:
# Once the training is completed, embedding layer has learnt the weights which are 
# nothing but the vector representations of each word. Lets check the shape of the weight matrix.

print(model.layers[0].get_weights()[0].shape)

# As we can see from the result of this statement, the shape of the wt matrix of the trained embedding layer is 10000 * 8
# which is consistent with -> 10000 unique words which we fed the model AND 8 being the length of each word vector embedding 

(10000, 8)


### Using pre-trained word embeddings
